In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input 
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import cv2
import os


def predict_person(model_classification, path):
  labels = ['personA', 'personB', 'personC', 'personD', 'personE']
  image = load_img(path, target_size=(224, 224))
  img = np.array(image)
  img = img / 255.0
  img = img.reshape(1,224,224,3)
  VGG_model = load_model(model_classification)
  predictions = VGG_model.predict(img)
  person = labels[predictions.argmax()]
  return person



def detect_similarity(model_similarity, real_img, img, threshold=0.5):
  image1 = cv2.imread(real_img)
  image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
  image1 = cv2.resize(image1, (128, 128))

  image2 = cv2.imread(img)
  image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
  image2 = cv2.resize(image2, (128, 128))

  im1 = preprocess_input(image1)
  im2 = preprocess_input(image2)

  model = load_model(model_similarity)
  embedding1 = model.predict(np.array([im1]))
  embedding2 = model.predict(np.array([im2]))
  distance = np.sum(np.square(embedding1-embedding2), axis=-1)
  
  L = ['Not-Similar','Similar']
  prediction = np.where(distance<=threshold, 1, 0)
  
  return L[prediction[0]]


image_Path_for_classification = r'/content/drive/MyDrive/CV_Dataset/CV2023TestSamples/SignatureTestSamples/personA_forged.png'
model_classification = r'/content/drive/MyDrive/CV_Dataset/Saved models/model_VGG.h5'

real_image = r'/content/drive/MyDrive/CV_Dataset/Real_Data/personA/A.png'
model_similarity = r'/content/drive/MyDrive/CV_Dataset/Saved models/model.h5'

person = predict_person(model_classification, image_Path_for_classification)
print(person)

similarity = detect_similarity(model_similarity, real_image, image_Path_for_classification)
print(similarity)